In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from pymongo import MongoClient
import time

client = MongoClient("mongodb://localhost:27017")
db = client["test0912"]
collection = db["yeogi"]

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

url = "https://www.yeogi.com/domestic-accommodations?keyword=%EA%B0%95%EB%A6%89&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false&page=1"
driver.get(url)

wait = WebDriverWait(driver, 10)

for page_num in range(1, 6):
    print(f"\n===== {page_num} 페이지 데이터 =====")
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    urls = driver.find_elements(By.CSS_SELECTOR, "a.gc-thumbnail-type-seller-card")
    titles = driver.find_elements(By.CSS_SELECTOR, "div.css-1j7tt62 > div > h3")
    prices = driver.find_elements(By.CSS_SELECTOR, "span.css-5r5920")
    ratings = driver.find_elements(By.CSS_SELECTOR, "span.css-9ml4lz")
    places = driver.find_elements(By.CSS_SELECTOR, "span.css-ki0lqh")
    reviews = driver.find_elements(By.CSS_SELECTOR, "span.css-oj6onp")
    
    page_data = []
    
    for i in range(len(titles)):
        record = {
            "title": titles[i].text if i < len(titles) else None,
            "price": prices[i].text.replace(",", "") if i < len(prices) else None,
            "rating": ratings[i].text if i < len(ratings) else None,
            "place": places[i].text if i < len(places) else None,
            "review_count": reviews[i].text.replace("명 평가", "").replace(",", "") if i < len(reviews) else None,
            "url": urls[i].get_attribute("href") if i < len(urls) else None,
            "page": page_num
        }
        print(record)
        page_data.append(record)
    
    if page_data:
        collection.insert_many(page_data)
        print(f"👉 {len(page_data)}개 데이터 MongoDB 저장 완료!")

    if page_num == 5:
        break
    
    try:
        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='다음']")))
        next_btn.click()
        time.sleep(2)
    except Exception as e:
        print("다음 버튼 클릭 실패:", e)
        break

driver.quit()
client.close()


===== 1 페이지 데이터 =====
{'title': '★당일특가★ 세인트존스 호텔', 'price': '161820', 'rating': '9.1', 'place': '강릉 강문해변 앞', 'review_count': '9919', 'url': 'https://www.yeogi.com/domestic-accommodations/48951?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': '★당일특가★ 스카이베이호텔 경포', 'price': '209470', 'rating': '9.1', 'place': '강문해변 차량 2분', 'review_count': '6697', 'url': 'https://www.yeogi.com/domestic-accommodations/49688?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': '정동진 썬크루즈호텔', 'price': '198000', 'rating': '9.1', 'place': '정동진역 차량 5분', 'review_count': '2998', 'url': 'https://www.yeogi.com/domestic-accommodations/67769?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': 'SL 호텔 강릉', 'price': '105830', 'rating': '9.1', 'place': '주문진터미널 차량 4분', 'review_count': '2437', 'url': 'https://www.yeogi.com/domestic-accommodations/69744?checkIn=2025-09-12&checkOut=2025-09-13&personal=2', 'page': 1}
{'title': '강릉 로맨스 인 강문', 'price': '206692', 'r

In [16]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["test0912"]
collection = db["yeogi"]

pipeline = [
    {
        "$sort": {
            "rating": -1,
            "review_count": -1
        }
    },
    {
        "$limit": 10
    },
    {
        "$project": {
            "_id": 0,
            "title": 1,
            "rating": 1,
            "review_count": 1,
            "price": 1,
            "place": 1,
            "url": 1
        }
    }
]

results = list(collection.aggregate(pipeline))

print("\n=== 평점 상위 10 숙소(동률 시 리뷰 수 기준) ===")
for r in results:
    print(f"{r['title']} | 평점: {r['rating']} | 리뷰수: {r['review_count']} | 가격: {r['price']} | 위치: {r['place']} | URL: {r['url']}")

client.close()


=== 평점 상위 10 숙소 ===
강릉 라온 펜션 | 평점: 10.0 | 리뷰수: 176 | 가격: None | 위치: 강문해변 차량 3분 | URL: https://www.yeogi.com/domestic-accommodations/60202?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 오릿 | 평점: 10.0 | 리뷰수: 122 | 가격: 79000 | 위치: 소돌해변 도보 10분 | URL: https://www.yeogi.com/domestic-accommodations/81827?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 오리아뜰리에풀빌라펜션 | 평점: 10.0 | 리뷰수: 94 | 가격: 297000 | 위치: 강문해변 도보 2분 | URL: https://www.yeogi.com/domestic-accommodations/81245?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 경포 서지별담 독채 스테이 | 평점: 10.0 | 리뷰수: 52 | 가격: 62100 | 위치: 경포호 차량 2분 | URL: https://www.yeogi.com/domestic-accommodations/82852?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 스테이 이플 | 평점: 10.0 | 리뷰수: 40 | 가격: 260000 | 위치: 순포습지 도보 1분 | URL: https://www.yeogi.com/domestic-accommodations/76803?checkIn=2025-09-12&checkOut=2025-09-13&personal=2
강릉 피어하우스독채풀빌라 | 평점: 10.0 | 리뷰수: 14 | 가격: 30500 | 위치: 강문해변 차량 9분 | URL: https://www.yeogi.com/domestic-accommodations/83700?

In [ ]:
# Selenium 라이브러리에서 웹 브라우저를 자동으로 제어하기 위한 도구들을 불러옵니다.
from selenium import webdriver  # 실제 크롬 브라우저를 열고 조작할 수 있게 해주는 도구입니다.
from selenium.webdriver.chrome.service import Service  # ChromeDriver를 서비스 형태로 실행할 수 있도록 도와주는 모듈입니다.
from selenium.webdriver.chrome.options import Options  # 브라우저의 실행 옵션 (창 크기, 언어, 사용자 정보 등)을 설정할 수 있는 도구입니다.

# 웹 페이지 내에서 원하는 요소를 찾을 때 사용할 기준(예: 태그, 클래스 등)을 제공하는 모듈입니다.
from selenium.webdriver.common.by import By

# 웹 페이지가 완전히 로드되었는지 또는 특정 요소가 나타났는지를 기다릴 수 있는 도구입니다.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  # 특정 조건이 만족될 때까지 기다릴 수 있는 조건들을 정의합니다.

# 크롬 브라우저용 드라이버를 자동으로 다운로드하고 경로를 설정해주는 라이브러리입니다.
from webdriver_manager.chrome import ChromeDriverManager

# MongoDB라는 NoSQL 데이터베이스에 접속하고 데이터를 저장하거나 읽기 위한 라이브러리입니다.
from pymongo import MongoClient

# 코드 실행 중간에 일부러 잠깐 멈추고 기다리게 만들기 위한 표준 모듈입니다.
import time

client = MongoClient("mongodb://localhost:27017")  # 로컬 컴퓨터에서 실행 중인 MongoDB 서버에 연결합니다.
db = client["test0912"]  # "test0912"라는 이름의 데이터베이스에 접속합니다. 없다면 새로 만들어집니다.
collection = db["yeogi"]  # 해당 데이터베이스 안의 "yeogi"라는 이름의 컬렉션(표 같은 개념)에 접속합니다.

service = Service(ChromeDriverManager().install())  # 최신 버전의 ChromeDriver를 자동으로 설치하고 실행을 위한 Service 객체를 생성합니다.


options = Options()  # 크롬 실행 옵션을 설정할 수 있는 객체를 생성합니다.
options.add_argument("--window-size=1920x1080")  # 브라우저 창의 크기를 1920x1080으로 설정합니다.
options.add_argument("--start-maximized")  # 브라우저를 실행하자마자 최대화된 상태로 띄웁니다.
options.add_argument("--user-agent=Mozilla/5.0...")  # 웹사이트에서 봤을 때 봇이 아닌 일반 브라우저처럼 보이도록 사용자 정보를 설정합니다.
options.add_argument("--lang=ko_KR")  # 웹사이트가 다국어를 지원하는 경우, 한국어 페이지가 보이도록 언어 설정을 한국어로 지정합니다.

driver = webdriver.Chrome(service=service, options=options)  # 위에서 설정한 옵션과 함께 Chrome 브라우저를 실행합니다.

# 여기어때에서 "강릉"이라는 키워드로 숙소를 검색한 URL입니다.
url = "https://www.yeogi.com/domestic-accommodations?keyword=%EA%B0%95%EB%A6%89&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false&page=1"
driver.get(url)  # 해당 URL로 브라우저를 이동시킵니다.

wait = WebDriverWait(driver, 10)  # 최대 10초 동안 특정 요소가 로딩될 때까지 기다릴 수 있는 객체를 만듭니다.


for page_num in range(1, 6):  # 1페이지부터 5페이지까지 반복하면서 크롤링을 진행합니다.
    print(f"\n===== {page_num} 페이지 데이터 =====")  # 현재 몇 페이지를 처리 중인지 출력합니다.


    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 자바스크립트를 이용해 웹 페이지 가장 아래로 스크롤합니다.
    time.sleep(2)  # 스크롤 후에 동적으로 데이터가 로드되도록 2초간 잠시 대기합니다.

    urls = driver.find_elements(By.CSS_SELECTOR, "a.gc-thumbnail-type-seller-card")  # 숙소 상세 페이지로 이동하는 링크 요소들을 수집합니다.
    titles = driver.find_elements(By.CSS_SELECTOR, "div.css-1j7tt62 > div > h3")  # 숙소의 이름(제목) 요소들을 수집합니다.
    prices = driver.find_elements(By.CSS_SELECTOR, "span.css-5r5920")  # 숙소의 가격 정보를 담은 요소들을 수집합니다.
    ratings = driver.find_elements(By.CSS_SELECTOR, "span.css-9ml4lz")  # 숙소의 평점을 담고 있는 요소들을 수집합니다.
    places = driver.find_elements(By.CSS_SELECTOR, "span.css-ki0lqh")  # 숙소의 위치(예: 강릉시 옥계면)를 나타내는 요소들을 수집합니다.
    reviews = driver.find_elements(By.CSS_SELECTOR, "span.css-oj6onp")  # 리뷰 수를 나타내는 텍스트 요소들을 수집합니다.

    page_data = []  # 한 페이지에 있는 모든 숙소 정보를 담을 리스트입니다.

    for i in range(len(titles)):  # 수집된 숙소 제목의 수만큼 반복하면서 각 숙소 정보를 처리합니다.
        record = {
            "title": titles[i].text if i < len(titles) else None,  # 숙소 이름 텍스트
            "price": prices[i].text.replace(",", "") if i < len(prices) else None,  # 가격 정보에서 쉼표 제거
            "rating": ratings[i].text if i < len(ratings) else None,  # 평점 정보 텍스트
            "place": places[i].text if i < len(places) else None,  # 위치 정보 텍스트
            "review_count": reviews[i].text.replace("명 평가", "").replace(",", "") if i < len(reviews) else None,  # 리뷰 수에서 "명 평가"와 쉼표 제거
            "url": urls[i].get_attribute("href") if i < len(urls) else None,  # 숙소 상세 페이지 링크
            "page": page_num  # 현재 몇 번째 페이지에서 수집된 데이터인지 기록
        }
        print(record)  # 콘솔에 현재 수집한 숙소 정보를 출력합니다.
        page_data.append(record)  # 해당 정보를 page_data 리스트에 추가합니다.

    if page_data:  # 수집된 데이터가 있을 경우에만 저장합니다.
        collection.insert_many(page_data)  # 여러 개의 데이터를 한 번에 MongoDB 컬렉션에 저장합니다.
        print(f"👉 {len(page_data)}개 데이터 MongoDB 저장 완료!")  # 저장한 개수를 콘솔에 출력합니다.


    if page_num == 5:  # 5페이지까지 수집했으면 반복 종료
        break

    try:
        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='다음']")))  # "다음" 버튼이 클릭 가능해질 때까지 기다립니다.
        next_btn.click()  # 클릭해서 다음 페이지로 이동합니다.
        time.sleep(2)  # 페이지 전환 후 2초 대기
    except Exception as e:
        print("다음 버튼 클릭 실패:", e)  # 예외가 발생하면 에러 메시지를 출력하고 반복을 멈춥니다.
        break

driver.quit()  # 크롬 브라우저를 완전히 종료합니다.
client.close()  # MongoDB와의 연결을 종료합니다.